# Scraping Medical Research Topics on TED Talks


## About Web Scraping


Data analysts analyze and report insights gleaned from data. Ever wondered how these data is collected? That's where knowledge of web scraping comes in handy.

### What is web scraping?

Web scraping is a technique used to extract content and data from websites.The data extracted is stored in databases and retrieved later to perform analysis and communicate insights.

*“Data are just summaries of thousands of stories.”* **– By Chip & Dan Heath**

Given that large amounts of data is extracted, web scraping automates tasks that might otherwise take humans far longer, or even be impossible to complete on a timely basis.

How does web scraping work?

Hypertext Markup Language(HTML) is used to give structure to websites. A mark-up language is universal meaning scrapers can easily pinpoint specific elements within them and extract content.

A general process of web scraping follows these steps:

- Identifying a site to scrape.
- Use `Request` to fetch the HTML code.
- Locate HTML elements using Beautiful Soup.
- Use Pandas to create CSV  files.


## About TED Talks:

TED Conferences LLC(Technology, Entertainment, Design) is an American media organization that posts talks online for free distribution under the slogan "ideas worth spreading".TED's early emphasis was on technology and design, consistent with its Silicon Valley origins. It has since broadened its perspective to include talks on many scientific, cultural, political, humanitarian and academic topics.

In this project, we are going to scrape inspirational medical research topic presented on Ted Talks.


![Title](https://i.ibb.co/PDPrnVW/ashraful-islam-p-Rt3-JVYl-Jho-unsplash.jpg)


Photo by <a href="https://unsplash.com/@ashraful25?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Ashraful Islam</a> on <a href="https://unsplash.com/photos/a-double-strand-of-blue-and-white-spirals-pRt3JVYlJho?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>
      

## Website of Interest and Objectives

Our website of interest is https://www.ted.com/talks?page=1&sort=popular&topics%5B%5D=medical+research

As earlier mentioned, this site contains talks. From the talks posted we want to extract the following information.
- Title of the talk
- The speaker
- Talk URL
- Duration of the talk
- Talk views



## Web Scraping Toolbox

Before we proceed to the next step, let's learn a little bit about the tools that we will use for web scraping. We will make use of three python libraries for this project.

`Requests`
- for making various types of HTTP requests.

`Beautiful Soup`
- for pulling data out of HTML and XML files.

`Pandas`
- for storing the data in the required format.

## Download Web Pages Using `Requests`

We will use the `requests` library to fetch HTML code from the website.

In [65]:
# Installing the library
!pip install requests --upgrade --quiet

In [66]:
# import the library
import requests

In [67]:
#variable for our website of interest
talks_url = 'https://www.ted.com/talks?page=1&sort=popular&topics%5B%5D=medical+research'

We will use `requests.get` function to make a request from the webpage. A `response` object is generated once Requests gets a response back from the server.

In [68]:
response = requests.get(talks_url)

We can check if our request is succesful using the `response.status_code`, the values of which is set between 200 and 299.

In [69]:
response.status_code

200

We can access the contents of the webpage using the `.text` property of the `response`.

In [70]:
page_content = response.text

Let's take a peak of the page_content.

In [71]:
len(page_content)

175586

The page has 175,586 characters.

In [72]:
page_content[:1000]

'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta property="og:url" content="https://www.ted.com/talks"/><link rel="canonical" href="https://www.ted.com/talks"/><meta name="title" content="TED Talks"/><meta name="description" content="TED Talks are influential videos from expert speakers on education, business, science, tech and creativity, with subtitles in 100+ languages. Ideas free to stream and download."/><meta name="keywords" content="TED, Talks, Themes, Speakers, Technology, Entertainment, Design"/><meta property="og:image" content="https://pi.tedcdn.com/r/pl.tedcdn.com/social/ted-logo-fb.png?v=wAff13s?"/><meta property="og:title" content="TED Talks"/><meta property="og:description" content="TED Talks are influential videos from expert speakers on education, business, science, tech and creativity, with subtitles in 100+ languages. Ideas free to stream and download."/><meta property="og:url" content="https://www.ted.com/talks"/><title>TED: Ideas change everything

Above is the `source code` of the webpage written in HTML.

Let's save the contents to a file with the `.html` extension.

In [73]:
with open('talks.html' , 'w') as file:
    file.write(page_content)

## Beautiful Soup : Parse and Extract Information

To use Beautiful Soup, we first need to install it

In [74]:
#Install library
!pip install beautifulsoup4 --upgrade --quiet

In [75]:
#import library
from bs4 import BeautifulSoup

Then, we create the Beautiful Soup object with the `page_content` as the input.

In [76]:
talk_doc = BeautifulSoup(page_content, 'html.parser')

In [77]:
type(talk_doc)

bs4.BeautifulSoup

We have used a lot of code to prepare our web document for scraping. Let us now create a function to download the page.

In [78]:
import requests
from bs4 import BeautifulSoup
def get_page(talks_url):
    #URL to scrape
    talks_url = talks_url
    # get HTML page using requests
    response = requests.get(talks_url)
    #confirm request is a success
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(talks_url))
    #create beautiful soup object
    talk_doc = BeautifulSoup(response.text)
    return talk_doc

In [79]:
type(talk_doc)

bs4.BeautifulSoup

Data is stored in HTML tags. Beautiful soup has a lot of methods of searching through these tags, https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree. The two common ones being `find()` and `find_all()`. For our project we will use the `find_all()` method to extract information from our webpage and return lists.

Let's now create some functions using the `find_all()` method.

We will first need to inspect our page. Below we have an image of the webpage at the top. The HTML code which gives the page structure is at the bottom.

We can get the image by right clicking on our image of interest and selecting the option inspect element.

![Title](https://i.ibb.co/j8Jz9y9/Annotation-2021-05-10-140602.png)

The speakers names, talk urls, talk title, the number of views and the talks duration are among the information we need to extract from the webpage.
If you are familiar with HTML, you will notice that some of the information that we need to extract from the web page are located within a h4 tag which has different classes.

We will now create functions to extact these information.

In [80]:
#Talk titles
def get_title(talk_doc):
    # Initialize an empty list to store talk titles
    title = []

    # Define the class used for talk titles in <h4>
    selection_class = ('f-w:700 h9 m5')

    # Search for all <h4> tags with the specific class
    title_tags = talk_doc.find_all('h4', {'class': selection_class})

    # Loop through each <h4> tag that contain the talk title
    for tag in title_tags:
        # Append the stripped text (removing leading/trailing spaces) to the title list
        title.append(tag.text.strip())

    # Return the list of extracted titles
    return title



In [81]:
#Check if our function is working
get_title(talk_doc)

[]

Our function is working properly. Let's move to the next.

In [82]:
#Speakers names
def get_speakers(talks_doc):
    speakers= []
    speaker_class = 'h12 talk-link__speaker'
    speaker_tag = talks_doc.find_all('h4',{'class' : speaker_class })
    for tag in speaker_tag:
        speakers.append(tag.text)

    return speakers

In [83]:
get_speakers(talk_doc)

[]

In [84]:
#Talk duration
def get_time(talk_doc):
    talk_time = []
    duration_tag = 'thumb__duration'
    talk_duration = talk_doc.find_all('span', {'class' : duration_tag })
    for duration in talk_duration:
        talk_time.append(duration.text)

    return talk_time

In [85]:
get_time(talk_doc)

[]

In [86]:
#Talk view
def get_views(talk_doc):
    talk_views = []
    views_class = 'meta__val'
    views = talk_doc.find_all('span',{'class' : views_class})
    for view in views:
        talk_views.append(view.text.strip())

    return talk_views

In [87]:
get_views(talk_doc)

[]

In [88]:
#Talks urls
def get_urls(talk_doc):
    urls = []
    a_tags = talk_doc.find_all('a', {'class': 'ga-link', 'data-ga-context': 'talks'})
    for a_tag in a_tags:
        url = ('https://www.ted.com/' + a_tag['href'])
        if url not in urls:
            urls.append('https://www.ted.com/' + a_tag['href'])

    return urls

In [89]:
get_urls(talk_doc)

[]

## Create CSV Files Using Pandas

In [90]:
#Installing Pandas library
!pip install pandas --quiet

In [91]:
#Import Pandas
import pandas as pd

In [92]:
#Create a dictionary
talks_dict = {
    'Title' : get_title(talk_doc),
    'Speaker' : get_speakers(talk_doc),
    'Duration' : get_time(talk_doc),
    'Views' : get_views(talk_doc),
    'URLs' : get_urls(talk_doc),}

In [93]:
talks_dict

{'Title': [], 'Speaker': [], 'Duration': [], 'Views': [], 'URLs': []}

In [94]:
#Create dataframe
talks_pd = pd.DataFrame(talks_dict)

In [95]:
talks_pd

,Title,Speaker,Duration,Views,URLs


We have now extracted information from the first page of the webpage.

In [96]:
#Save in a csv file
talks_pd.to_csv('talks.csv', index = None)